## Web Scraping Nike

In [55]:
# Importando bibliotecas 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re

In [3]:
# verificando quantas páginas temos para extrair as informações
url = 'https://www.nike.com.br/busca?term=tenis&page=1'
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=header)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

info_pags = soup.find('span', {'class': 'Pagination-styled__MobilePageStatus-sc-7faf39a5-5 bbwvsP'})
text_pag = info_pags.get_text(strip=True)
last_pag = text_pag.split('de')[-1]

# ultima página
last_pag = int(last_pag.strip())

In [4]:
# Função para extrair informações da página

def extract_data(url):

# requisição na página com user agent
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=header)
    response = response.content

# trazendo a resposta da página para html
    soup = BeautifulSoup(response, 'html.parser')

# acessando a div que contém as informações sobre os produtos
    div_produto = soup.find('div', {'data-testid': 'products-search-3'})
    div_produto_lista = div_produto.contents

# criando uma lista para armazenar os dados dos produtos
    dados_pagina = []

# iterando sobre cada produto para trazer informação de nome, categoria, preço atual e preço
    for div_produto in div_produto_lista:
        div_produto_informacoes = div_produto.find(
            'div', {'class': 'ProductCard-styled__ProductContentContainer-sc-c594f81c-3 dKaYgI'})

        if div_produto_informacoes:
            div_produto_informacoes = div_produto_informacoes.contents

            nome_tenis = div_produto_informacoes[0].text
            categoria_tenis = div_produto_informacoes[1].text

            preco_tenis = div_produto_informacoes[2].text

            precos_encontrados = re.findall(r'R\$\s*(\d+,\d{2})', preco_tenis)

            if len(precos_encontrados) >= 2:
                preco_atual_tenis = precos_encontrados[0]
                preco_anterior_tenis = precos_encontrados[1]
            else:
                preco_atual_tenis = re.sub(
                    r'[^\d,]', '', preco_tenis).replace(',', '.')
                preco_anterior_tenis = 'null'

            dados_pagina.append(
                [nome_tenis, categoria_tenis, preco_atual_tenis, preco_anterior_tenis])

    return dados_pagina


In [5]:
# Loop pelas páginas 1 a last pag
dados_paginas = []

for page in range(1, last_pag+1):
    url = f'https://www.nike.com.br/busca?term=tenis&page={page}'
    dados_pagina = extract_data(url)
    dados_paginas.extend(dados_pagina)

##  Analise referente ao portfólio de tênis da Nike

In [15]:
# dataframe
colunas = ['Nome', 'Categoria', 'Preço Atual', 'Preço Anterior']
df = pd.DataFrame(dados_paginas, columns=colunas)


In [20]:
# tranformando coluna Preço Atual em númerica
df['Preço Atual'] = df['Preço Atual'].str.replace(',','.')
df['Preço Atual'] = df['Preço Atual'].str.extract(r'(\d+\.\d{2})')
df['Preço Atual'] = df['Preço Atual'].astype('float64')



In [ ]:
# tranformando coluna Preço Anterior em númerica
df['Preço Anterior'] = df['Preço Anterior'].replace('null',np.nan)
df['Preço Anterior'] = df['Preço Anterior'].str.replace(',','.')
df['Preço Anterior'] = df['Preço Anterior'].str.extract(r'(\d+\.\d{2})')
df['Preço Anterior'] = df['Preço Anterior'].astype('float64')

In [36]:
# criando coluna de desconto
df['Desconto'] = df['Preço Anterior']-df['Preço Atual']

df['% Desconto'] = df['Desconto']/df['Preço Anterior']


In [78]:
# visualizando df
df.head()
df.to_csv('tenis.csv')

In [107]:
# Analise Pareto - Categoria vs Quantidade de tênis
categoria_vs_qtd = df['Categoria'].value_counts().reset_index()

categoria_vs_qtd.columns = {'Categoria': 'Categoria', 'Quantidade': 'Quantidade'}
categoria_vs_qtd['% Acumulada'] = ( categoria_vs_qtd['Quantidade'].cumsum()/categoria_vs_qtd['Quantidade'].sum())*100
categorias_pareto = categoria_vs_qtd[categoria_vs_qtd['% Acumulada'] <= 80]

categorias_pareto

,Categoria,Quantidade,% Acumulada
0,Casual,541,38.314448
1,Corrida,220,53.895184
2,Treino & Academia,111,61.756374
3,Para Jogar Tênis,94,68.413598
4,Basquete,86,74.504249
5,Bebês / Casual,68,79.320113


In [106]:
# Top 10 Tênis com maiores descontos

top10_descontos = df.nlargest(10,'Desconto').reset_index()

top10_descontos[['Nome','Categoria','Preço Anterior','Desconto']]

,Nome,Categoria,Preço Anterior,Desconto
0,Tênis Nike Air Zoom Pegasus 39 Masculino,Corrida,999.99,500.0
1,Tênis Nike Air Zoom Pegasus 39 Masculino,Corrida,999.99,500.0
2,Tênis Nike SuperRep Go 3 Flyknit Next Nature F...,Treino & Academia,999.99,500.0
3,Tênis Nike SuperRep Go 3 NN Flyknit Masculino,Treino & Academia,999.99,490.0
4,Tênis NikeCourt Zoom Pro Premium Feminino,Para Jogar Tênis,999.99,480.0
5,Tênis Nike Air Zoom Pegasus 39 Feminino,Corrida,999.99,460.0
6,Tênis NikeCourt Zoom Pro Masculino,Para Jogar Tênis,999.99,460.0
7,Tênis NikeCourt Zoom Pro Masculino,Para Jogar Tênis,999.99,460.0
8,Tênis NikeCourt Air Zoom Pro Feminino,Para Jogar Tênis,999.99,460.0
9,Tênis Nike Air Max 90 Feminino,Casual,999.99,450.0


In [139]:
# Categorias com mais tênis em desconto

df_filtrado = df.dropna(subset=['Desconto'])


describe_categoria = df_filtrado.groupby('Categoria')['Desconto'].describe()

describe_categoria = describe_categoria.sort_values(by='count',ascending=False)

top_10_categorias = describe_categoria.nlargest(10,'count')


In [166]:
# verificando a % de tênis com desconto por categoria
categoria_desconto_vs_qtd = df_filtrado['Categoria'].value_counts().reset_index()
categoria_desconto_vs_qtd.columns = {'Categoria': 'Categoria', 'Quantidade com Desconto': 'Quantidade com Desconto'}

df_final = categoria_vs_qtd.merge(categoria_desconto_vs_qtd, on='Categoria', how='inner')
df_final = df_final.drop(columns=['% Acumulada'])

df_final['% com Desconto'] = (df_final['Quantidade com Desconto']/df_final['Quantidade']*100).round(2)

df_final

,Categoria,Quantidade,Quantidade com Desconto,% com Desconto
0,Casual,541,241,44.55
1,Corrida,220,92,41.82
2,Treino & Academia,111,54,48.65
3,Para Jogar Tênis,94,60,63.83
4,Basquete,86,45,52.33
5,Bebês / Casual,68,62,91.18
6,Pré-Adolescentes / Casual,62,37,59.68
7,Crianças / Casual,49,41,83.67
8,Pré-Adolescentes / Corrida,41,38,92.68
9,Crianças / Corrida,33,33,100.00
